In [ ]:
! pip install -q XlsxWriter

In [ ]:
!apt-get update
!apt-get install -y libreoffice fonts-ipafont poppler-utils
!pip install -q byaldi transformers accelerate qwen_vl_utils pdf2image bitsandbytes

In [ ]:
import xlsxwriter
import os

file_path = 'monthly_sales_report.xlsx'

# ワークブック作成
workbook = xlsxwriter.Workbook(file_path)
worksheet = workbook.add_worksheet('Sales Data')

# --- データと書式の準備 ---
header_fmt = workbook.add_format({'bold': True, 'bg_color': '#D7E4BC', 'border': 1})
data_fmt = workbook.add_format({'border': 1})
highlight_bad = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006', 'border': 1}) # 赤
highlight_good = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100', 'border': 1}) # 緑

# データ（4月に底を打ち、8月に回復するシナリオ）
headers = ['Month', 'Sales (万円)']
data = [
    ['1月', 400], ['2月', 350], ['3月', 300],
    ['4月', 200], # 底
    ['5月', 250], ['6月', 350], ['7月', 450],
    ['8月', 500], # 最高
]

# --- データの書き込み ---
worksheet.write_row('A1', headers, header_fmt)
for row_num, (month, sales) in enumerate(data, start=1):
    fmt = highlight_bad if sales == 200 else (highlight_good if sales == 500 else data_fmt)
    worksheet.write(row_num, 0, month, fmt)
    worksheet.write(row_num, 1, sales, fmt)

# --- グラフの作成 ---
chart = workbook.add_chart({'type': 'line'})
chart.add_series({
    'name':       ['Sales Data', 0, 1],
    'categories': ['Sales Data', 1, 0, 8, 0],
    'values':     ['Sales Data', 1, 1, 8, 1],
    'line':       {'color': 'blue', 'width': 2.25},
    'marker':     {'type': 'circle', 'size': 7},
})
chart.set_title ({'name': '2024年 上期売上推移'})
worksheet.insert_chart('D2', chart)

# --- テキストボックスの追加 ---
# ⚠️ これが重要です。通常のテキスト抽出では無視されます。
text_options = {
    'width': 250, 'height': 60,
    'font': {'bold': True, 'color': 'red', 'size': 10},
    'line': {'color': 'red', 'width': 2, 'dash_type': 'round_dot'},
    'align': {'vertical': 'middle', 'horizontal': 'center'},
}
worksheet.insert_textbox('H10', '※5月から新マーケティング施策を開始\nこれがV字回復の主要因です', text_options)

workbook.close()
print(f"✅ 生成完了: {file_path}")

In [ ]:
import subprocess
from pdf2image import convert_from_path
import matplotlib.pyplot as plt

def convert_excel_to_rag_ready(excel_path):
    """
    ExcelファイルをLibreOfficeでPDF化し、画像リストとして返す関数
    """
    output_dir = os.path.dirname(os.path.abspath(excel_path))
    filename = os.path.basename(excel_path)
    name_without_ext = os.path.splitext(filename)[0]

    print(f"🔄 Converting Excel to PDF: {filename}...")

    # LibreOfficeをヘッドレスモードで実行
    cmd = ["libreoffice", "--headless", "--convert-to", "pdf", "--outdir", output_dir, excel_path]
    try:
        subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    except subprocess.CalledProcessError as e:
        print(f"⚠️ Conversion failed: {e}"); return None, None

    pdf_path = os.path.join(output_dir, f"{name_without_ext}.pdf")
    if not os.path.exists(pdf_path): return None, None

    print(f"📄 PDF Generated: {pdf_path}")
    images = convert_from_path(pdf_path) # 画像化
    return pdf_path, images

# --- 実行と確認 ---
if os.path.exists(file_path):
    pdf_path, excel_images = convert_excel_to_rag_ready(file_path)

    # 👁️ AIが見る世界を可視化（グラフや注釈が見えるか確認！）
    if excel_images:
        plt.figure(figsize=(10, 6))
        plt.imshow(excel_images[0])
        plt.axis('off')
        plt.title("【AI Vision】What the model sees")
        plt.show()

In [ ]:
from byaldi import RAGMultiModalModel

# 🧠 ColPaliモデルのロード (verbose=0でログ抑制)
print("🚀 Loading ColPali model...")
RAG = RAGMultiModalModel.from_pretrained("vidore/colpali-v1.2", verbose=0)

# 📑 インデックス作成（画像Embedding）
RAG.index(
    input_path=pdf_path,
    index_name="excel_vision_index",
    store_collection_with_index=True,
    overwrite=True
)

# 🔍 検索クエリ：グラフの形状や視覚的特徴を問う
query = "売上の推移グラフを見て、最も落ち込んでいる月とその原因と思われる注釈を教えて"

print(f"🔍 Searching for: {query}")
results = RAG.search(query, k=1)

# ヒットした画像の取得
hit_page_idx = results[0].page_num - 1
retrieved_image = excel_images[hit_page_idx]

print(f"✅ Found relevant sheet (Page {results[0].page_num})")

In [ ]:
import gc
import torch

print("🧹 Cleaning up VRAM...")

# Retrieverモデルを削除
del RAG
del results

# ガベージコレクションとGPUキャッシュクリア
gc.collect()
torch.cuda.empty_cache()

print("✨ VRAM Cleaned! Ready for Generator.")

In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor, BitsAndBytesConfig
from qwen_vl_utils import process_vision_info

# 💾 メモリ不足対策：4bit量子化の設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model_name = "Qwen/Qwen2-VL-7B-Instruct"
print(f"🚀 Loading VLM (4-bit): {model_name}")

# モデルのロード
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
)
processor = AutoProcessor.from_pretrained(model_name)

# 📝 推論プロンプト
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": retrieved_image}, # 検索されたExcel画像
            {"type": "text", "text": f"画像の内容に基づき、次の質問に答えてください: {query}"},
        ],
    }
]

# 入力データの準備
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
).to("cuda")

print("🤖 Generating Answer...")

# 生成実行
generated_ids = model.generate(**inputs, max_new_tokens=512)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

print("\n" + "="*30)
print("🤖 AI Analysis Result")
print("="*30)
print(output_text[0])
